# SQLAlchemy

In [1]:
# Import all necessary modules

import sqlalchemy as sa
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import scoped_session, sessionmaker, relationship, backref, joinedload, selectinload, subqueryload

engine = sa.create_engine("sqlite:///:memory:")
Base = declarative_base()

In [2]:
# Declare All Models

class Author(Base):
    __tablename__ = "authors"
    id = sa.Column(sa.Integer, primary_key=True)
    name = sa.Column(sa.String)

    def __repr__(self):
        return "<Author(name={self.name!r})>".format(self=self)

class LazyBook(Base):
    __tablename__ = "lazybooks"

    id = sa.Column(sa.Integer, primary_key=True)
    title = sa.Column(sa.String)
    
    author_id = sa.Column(sa.Integer, sa.ForeignKey("authors.id"))
    author = relationship("Author", backref=backref("books1"), lazy=True)
    
    def __str__(self):
        return f"Book using {__tablename__}"

class NotLazyBook(Base):
    __tablename__ = "notlazybooks"
    
    id = sa.Column(sa.Integer, primary_key=True)
    title = sa.Column(sa.String)
    
    author_id = sa.Column(sa.Integer, sa.ForeignKey("authors.id"))
    author = relationship("Author", backref=backref("books2"), lazy=False)
    
class LazyRaiseBook(Base):
    __tablename__ = "lazyraisebooks"
    
    id = sa.Column(sa.Integer, primary_key=True)
    title = sa.Column(sa.String)
    
    author_id = sa.Column(sa.Integer, sa.ForeignKey("authors.id"))
    author = relationship("Author", backref=backref("books3"), lazy="raise")

class LazyNoLoadBook(Base):
    __tablename__ = "lazynoloadbooks"
    
    id = sa.Column(sa.Integer, primary_key=True)
    title = sa.Column(sa.String)
    
    author_id = sa.Column(sa.Integer, sa.ForeignKey("authors.id"))
    author = relationship("Author", backref=backref("books4"), lazy="noload")

class BoringLazyRaiseBook(Base):
    __tablename__ = "verylazyraisebooks"
    
    id = sa.Column(sa.Integer, primary_key=True)
    bore_index = sa.Column(sa.Integer)
    
    orig_book_id = sa.Column(sa.Integer, sa.ForeignKey("lazyraisebooks.id"))
    orig_book = relationship("LazyRaiseBook", backref=backref("lazy_back"), lazy="raise")


# Create all metadata
Base.metadata.create_all(engine)

Books = [LazyBook, NotLazyBook, LazyRaiseBook, LazyNoLoadBook]

In [3]:
# Seed Data
session = scoped_session(sessionmaker(bind=engine))

author1 = Author(name="Dennis Ritchie")
author2 = Author(name="Frank Hoff")

session.add(author1)
session.add(author2)

for Book in Books:
    b1 = Book(title="C", author=author1)
    b2 = Book(title="D", author=author2)
    b3 = Book(title="E", author=author1)
    b4 = Book(title="F", author=author2)
    session.add(b1)
    session.add(b2)
    session.add(b3)
    session.add(b4)

    if Book is LazyRaiseBook:
        vb1 = BoringLazyRaiseBook(bore_index=100, orig_book=b1)
        session.add(vb1)

session.commit()

In [4]:
# Query Data

for Book in Books:
    print(f"===============================")
    print(f"Query using {Book}: ")
    try:
        books = session.query(Book).all()
        for book in books:
            print(f"{book.title} {book.author_id} {book.author.name}")
        continue
    except Exception as e:
        print(f"Eror while querying {Book} {e}")

    print(f"\nJoinedLoad Query using {Book}: ")
    try:
        books = session.query(Book).options(joinedload(Book.author)).all()
        for book in books:
            print(f"{book.title} {book.author.name}")
        continue
    except Exception as e:
        print(f"Error even during JoinedLoad querying {Book} {e}")

    print(f"\nSelectinload Query using {Book}: ")
    try:
        books = session.query(Book).options(selectinload(Book.author)).all()
        for book in books:
            print(f"{book.title} {book.author.name}")
        continue
    except Exception as e:
        print(f"Error even during Selectinload querying {Book} {e}")

Query using <class '__main__.LazyBook'>: 
C 1 Dennis Ritchie
D 2 Frank Hoff
E 1 Dennis Ritchie
F 2 Frank Hoff
Query using <class '__main__.NotLazyBook'>: 
C 1 Dennis Ritchie
D 2 Frank Hoff
E 1 Dennis Ritchie
F 2 Frank Hoff
Query using <class '__main__.LazyRaiseBook'>: 
Eror while querying <class '__main__.LazyRaiseBook'> 'LazyRaiseBook.author' is not available due to lazy='raise'

JoinedLoad Query using <class '__main__.LazyRaiseBook'>: 
C Dennis Ritchie
D Frank Hoff
E Dennis Ritchie
F Frank Hoff
Query using <class '__main__.LazyNoLoadBook'>: 
Eror while querying <class '__main__.LazyNoLoadBook'> 'NoneType' object has no attribute 'name'

JoinedLoad Query using <class '__main__.LazyNoLoadBook'>: 
Error even during JoinedLoad querying <class '__main__.LazyNoLoadBook'> 'NoneType' object has no attribute 'name'

Selectinload Query using <class '__main__.LazyNoLoadBook'>: 
Error even during Selectinload querying <class '__main__.LazyNoLoadBook'> 'NoneType' object has no attribute 'name'


In [5]:
# LazyRaiseBook.author , "orig_book.author"
books = session.query(BoringLazyRaiseBook)\
        .options(joinedload(BoringLazyRaiseBook.orig_book)\
                 .subqueryload(LazyRaiseBook.author))
print(f"\n=========================================\nQuery is {books}")
for book in books.all():
            print(f"{book.bore_index} {book.orig_book.title} {book.orig_book.author.name}")

books = session.query(BoringLazyRaiseBook)\
        .options(joinedload("orig_book"))\
        .options(joinedload("orig_book.author"))
print(f"\n=========================================\nQuery is {books}")
for book in books.all():
            print(f"{book.bore_index} {book.orig_book.title} {book.orig_book.author.name}")


Query is SELECT verylazyraisebooks.id AS verylazyraisebooks_id, verylazyraisebooks.bore_index AS verylazyraisebooks_bore_index, verylazyraisebooks.orig_book_id AS verylazyraisebooks_orig_book_id, lazyraisebooks_1.id AS lazyraisebooks_1_id, lazyraisebooks_1.title AS lazyraisebooks_1_title, lazyraisebooks_1.author_id AS lazyraisebooks_1_author_id 
FROM verylazyraisebooks LEFT OUTER JOIN lazyraisebooks AS lazyraisebooks_1 ON lazyraisebooks_1.id = verylazyraisebooks.orig_book_id
100 C Dennis Ritchie

Query is SELECT verylazyraisebooks.id AS verylazyraisebooks_id, verylazyraisebooks.bore_index AS verylazyraisebooks_bore_index, verylazyraisebooks.orig_book_id AS verylazyraisebooks_orig_book_id, authors_1.id AS authors_1_id, authors_1.name AS authors_1_name, lazyraisebooks_1.id AS lazyraisebooks_1_id, lazyraisebooks_1.title AS lazyraisebooks_1_title, lazyraisebooks_1.author_id AS lazyraisebooks_1_author_id 
FROM verylazyraisebooks LEFT OUTER JOIN lazyraisebooks AS lazyraisebooks_1 ON lazyrai